In [ ]:
import  pandas as pd 
import  numpy as np
import  matplotlib.pyplot as plt
import  seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from matplotlib.ticker import MaxNLocator

### A [Chemometrics](https://en.wikipedia.org/wiki/Chemometrics) example from Multi- and Megavariate Data Analysis by Umetrics:
### A sugar plant in Scandinavia needs to measure the ash content of the sugar, an important quality parameter. Traditional wet-chemistry techniques are intrusive and time-consuming. The company is exploring the use of an on-line fluorescent spectroscopy method.
### 106 fluorescence measurements (time points) were taken during the 2.5 days start-up period of the plant, resulting in 571 discrete wavelengths (or X variables). For the multivariate calibration of our model, two wet-chemistry outputs Y were measured (i.e. V1_2 and V2_2).
### Let's take a look at the raw data. 
- The first 2 columns are metadata and can be ignored.
- Columns V1-V571 are the spectral data that are used as input X
- The last 2 columns V1_2 and V2_2 are the outputs Y we want to predict. Here, we show how to predict V1_2

<img src="Fig1_Intro_data.png"
     style="float: left; margin-right: 10px; width: 700px" />

In [ ]:
sugar = pd.read_excel('Sugar.xlsx')
sugar

### Slice based on index only the spectral input X, and plot the 106 spectra

In [ ]:
sugar_spectra = sugar.iloc[:,2:573]
sugar_spectra

In [ ]:
sugar_spectra.transpose().plot(legend=None,
                              figsize=(8,6),
                              title='Raw fluorescence data of 106 data points (lines)',
                              xlabel='Wavelength',
                              ylabel='Signal')

### Can we condense the information of each time-point or spectrum (or row) down to let's say 2-dimensions?
### For this, we will perform PCA on the X data. This is also an EDA method. 
### Spectral inputs are pre-processed (mean-centered) and fit to a 2 principal components model

In [ ]:
scaler = StandardScaler(with_std=False)
scaler

In [ ]:
sugar_spectra_scaled = scaler.fit_transform(sugar_spectra)
# sugar_spectra_scaled # To view the transformed data
# sugar_spectra_scaled.mean(axis=0) # To confirm that the mean of each of the 571 variables X is 0

In [ ]:
pca = PCA(n_components=2, svd_solver='full')

In [ ]:
pca_scores = pca.fit_transform(sugar_spectra_scaled)

In [ ]:
scores_pd = pd.DataFrame(data = pca_scores
                         ,columns = ['PC1', 'PC2']
                         ,index = sugar_spectra.index)
scores_pd

In [ ]:
loadings_pd = pd.DataFrame(data = pca.components_.T
                           ,columns = ['PC1', 'PC2']
                           ,index = sugar_spectra.columns)
loadings_pd

### Recap of the PCA transformation, dimension reduction method

<img src="Fig_2_PCA.png"
     style="float: left; margin-right: 10px; width: 700px" />

### Let's plot the scores to identify possible trends, outliers, clusters.

In [ ]:
def score_plot(scores, score_labels=None):
    # adjusting the scores to fit in (-1,1)
    xt = scores[:,0]
    yt = scores[:,1]
    scalext = 1.0/(xt.max() - xt.min())
    scaleyt = 1.0/(yt.max() - yt.min())
    xt_scaled = xt * scalext
    yt_scaled = yt * scaleyt
    
    fig = plt.figure(figsize=(9, 9))
    for (x,y), label in zip(np.vstack((xt_scaled, yt_scaled)).T, score_labels):
        plt.text(x, y, label, ha='center', size=11)
        
    plt.hlines(0, -1, 1, linestyles='solid', linewidth=3)
    plt.vlines(0, -1, 1, linestyles='solid', linewidth=3)
    
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.xlabel("PC{}".format(1), fontsize=20);
    plt.ylabel("PC{}".format(2), fontsize=20);
    plt.tick_params(labelsize=16)
    plt.grid()

In [ ]:
score_plot(pca_scores[:,:2], score_labels=scores_pd.index)
plt.show()

### The scores show an initial transient behaviour (points 0-15 will be discarded) and an outlier (point 87 will also be discarded)
### Let's plot the loadings to see what is the PCA picking as signal in the 2 components. Usually and for a dozen of variables we plot the loadings as vectors on the score plot. For spectroscopic data though it makes more sense to plot them as below.

In [ ]:
fig0 = plt.figure(figsize=(16, 6))
sub1 = fig0.add_subplot(121)
plt.plot(loadings_pd['PC1'],'-')
sub1.set_xlabel('Wavelength', fontsize=20)
sub1.set_ylabel('Loading value', fontsize=20)
sub1.set_title('Loadings of PC #1', fontsize=20)
sub1.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tick_params(labelsize=14)

sub2 = fig0.add_subplot(122)
plt.plot(loadings_pd['PC2'],'-')
sub2.set_xlabel('Wavelength', fontsize=20)
sub2.set_title('Loadings of PC #2', fontsize=20)
sub2.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tick_params(labelsize=14)

plt.show()

### The first PC captures predominantly information from the wavelengths that corresponds to V161-V401.
### PC2 captures information from the wavelengths between V50 and V150.
### Review the explained variance of the PCs.

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_*100

In [ ]:
fig = plt.figure(figsize=(9, 6))
sub0 = fig.add_subplot(111)
plt.plot(range(1,pca.n_components+1), np.cumsum(pca.explained_variance_ratio_),'b-s')
sub0.set_xlabel('Number of components', fontsize=20)
sub0.set_ylabel('Cumulative explained variance', fontsize=20)
sub0.set_title('Cumulative explained variance \n as a function of the number of components', fontsize=20)
sub0.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tick_params(labelsize=16)
plt.ylim(0,1)
plt.show()

### Remove the outliers, split in train/test data and fit the 3 PCs to a linear regression with the output V1_2

In [ ]:
X_spectra_scaled = pd.DataFrame(sugar_spectra_scaled)
X_spectra_scaled = X_spectra_scaled.iloc[15:].drop(X_spectra_scaled.index[87])


In [ ]:
y = sugar.iloc[15:, -2].drop(scores_pd.index[87])

### With the 2 Principal Components in the **scores** table capturing 99.1% of the variance in the X data, one may think to use these two variables (or 2PCs) in a linear regression model to find the relationship between the 2 PCs and the output Y. This is called **Principal Component Regression** or [PCR](https://scikit-learn.org/dev/auto_examples/cross_decomposition/plot_pcr_vs_pls.html). Although this is not always the best solution (why? next), we strongly suggest you try that.
### **Why** is PCR, or coupling the PCA of inputs X to a regression with the output Y, **not the best solution**?
### Keep in mind that PCA is an unsupervised method. It does not look at the output Y. It simply finds the directions with the maximum variance. If the output Y is strongly correlated with low variance directions, then the regression will not be very efficient.
### In contrast, **[cross decomposition or latent variable approaches](https://scikit-learn.org/stable/modules/cross_decomposition.html#cross-decomposition)** model the relationship between X and Y 

### Next, use Sklearn's cross-decomposition function [PLSRegression](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_spectra_scaled, y, test_size=1/2, random_state=23)

In [ ]:
pls_sklearn = PLSRegression(n_components=2, scale=False)
pls_sklearn.fit(X_train, y_train)

In [ ]:
score_plot(pls_sklearn.x_scores_[:,:2], score_labels=scores_pd.index)
plt.show()

In [ ]:
pls_sklearn.score(X_train,y_train)

### Compare the train/test predictions vs the actual values

In [ ]:
y_train_predicted = pls_sklearn.predict(X_train)
y_test_predicted = pls_sklearn.predict(X_test)

In [ ]:
fig2 = plt.figure(figsize=(9, 6))
sub5 = fig2.add_subplot(111)
plt.scatter(y_train_predicted, y_train, s=60, marker="o", edgecolors='b')
plt.scatter(y_test_predicted, y_test, s=60, marker="o", edgecolors='r')
plt.legend(['Train set', 'Test set'])
plt.plot([0, 0.02], [0, 0.02])
plt.xlabel('Predicted', fontsize = 20)
plt.ylabel('Actual', fontsize = 20)
plt.tick_params(labelsize=16)
plt.xlim([0, 0.02])
plt.ylim([0, 0.02])

In [ ]:
error = y_test_predicted[:,0] - y_test
fig3 = plt.figure(figsize=(9, 6))
error.plot(kind='hist', title = 'Histogram of the PLS model error')

In [ ]:
print('The average error is: %.4f' % error.mean())
print('The average standard deviation is: %.4f' % error.std())
print('The minimum error is: %.4f' % error.min())
print('The maximum error is: %.4f' % error.max())

In [ ]:
fig4 = plt.figure(figsize=(9,6))
plt.scatter(y_test, error, s=80, marker="o", facecolors='none', edgecolor='black')
plt.hlines(0, 0, 0.02, linestyles='solid')
plt.title('Residual errors', fontsize = 20)
plt.xlabel('Output', fontsize = 20)
plt.ylabel('Model error', fontsize = 20)
plt.tick_params(labelsize=16)
plt.xlim([0, 0.02])
plt.ylim([-0.02, 0.02])